In [17]:
#Importing needed libraries
#import cx_Oracle
import pandas as pd
#import pyodbc
#from datetime import datetime, timedelta, date
import requests
#import json
import os
from io import StringIO
import urllib.request as urllib2

import selenium
import getpass
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 

import win32com.client

#pd.set_option('display.max_columns', 30)
#pd.set_option('display.max_rows', 1000)
#pd.set_option('display.float_format', '{:.4f}'.format)

In [18]:
#Detecting user
user =  os.getlogin()
user

's830227'

In [19]:
#Detecting user mail (last user log in into Outlook) + asking DS password
outlook = win32com.client.Dispatch('outlook.application')
mapi = outlook.GetNamespace("MAPI")

for account in mapi.Accounts:
    email_account = account.DeliveryStore.DisplayName
email_account
mail_password = getpass.getpass()

········


In [20]:
#Paths
Dollar_Exposure_TC = "C:\\Users\\" + user + "\\Cargill Inc\\RMG Risk Engine - Documents\\General\\040_Design\\040 Reporting\\Common Report Folder\\040 Queries\\CSST_Dollar Exposure_SD_TC.csv"
PnL_TC = "C:\\Users\\" + user + "\\Cargill Inc\\RMG Risk Engine - Documents\\General\\040_Design\\040 Reporting\\Common Report Folder\\040 Queries\\CSST_PnL_SD_TC.csv"


In [21]:
#Open Chrome and log in into Risk Engine to extract token

def login(user, passwd):
    #Ids of the buttons userName, Password, Next button
    un_field = (By.ID, "i0116")
    pw_field = (By.ID, "i0118")
    next_button = (By.ID, "idSIButton9")

    #Wait till the userName field is available then send the userName
    WebDriverWait(driver, 20).until(EC.presence_of_element_located(un_field)).send_keys(user)

    #click on the Next button
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_button)).click()

    #Wait till the userName field is available then send the userName
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(pw_field)).send_keys(passwd)

    #click on the Login button, has the same ID as the next button.
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_button)).click()

    #click on the Login button, has the same ID as the next button.
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_button)).click()
    
    #Wait till the url is the dashboard
    WebDriverWait(driver, 20).until(EC.url_to_be("https://cargill-prod-aer.fisglobal.com/riskui/#/dashboard/home"))
    return driver

driver = webdriver.Chrome()
driver.get('https://cargill-prod-aer.fisglobal.com/riskui')
page = driver.page_source
print("Microsoft Log In")
wait = WebDriverWait(driver, 10000).until(EC.url_contains("https://login.microsoftonline.com/"))
try:
    print('Authorizing')
    login(email_account, mail_password)
except Exception as error:
    print('Error', error)
else:
    print('Authorized')
token = driver.execute_script("return window.sessionStorage.getItem('access_token');")
print("Token acquired")
print(token)
print('Closing Chrome')
driver.close()
print('Chrome Close')

Microsoft Log In
Authorizing
Authorized
Token acquired
eyJhbGciOiJSUzI1NiIsImtpZCI6IkJEQTFCODVBMzBCNTdDQjc3OEFBODE5NDQ5NUYzMjIxRDhFQjdGRkUiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJ2YUc0V2pDMWZMZDRxb0dVU1Y4eUlkanJmXzQifQ.eyJuYmYiOjE2NTY2NTk5MjYsImV4cCI6MTY1NjY2MzUyNiwiaXNzIjoiaHR0cHM6Ly9jYXJnaWxsLXByb2QtYWVyLmZpc2dsb2JhbC5jb20vYWRhcHRpdmF1dGhlbnRpY2F0aW9uc2VydmljZSIsImF1ZCI6WyJodHRwczovL2NhcmdpbGwtcHJvZC1hZXIuZmlzZ2xvYmFsLmNvbS9hZGFwdGl2YXV0aGVudGljYXRpb25zZXJ2aWNlL3Jlc291cmNlcyIsImFwaSJdLCJjbGllbnRfaWQiOiJDYXJnaWxsU0FNTCIsInN1YiI6IlNVRElQVE9fREVCQENSR0wtVEhJUkRQQVJUWS5DT00iLCJhdXRoX3RpbWUiOjE2NTY2NTk5MjQsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzU3MzY4YzIxLWI4Y2YtNDJjZi1iZDBiLTQzZWNkNGJjNjJhZS8iLCJzY29wZSI6WyJvcGVuaWQiLCJwcm9maWxlIiwiYXBpIiwib2ZmbGluZV9hY2Nlc3MiXX0.jsGt7nqe1Jae4FgtX68o3ryz74mEyKcoYOINYo6Z790OhkgOGBSSJMcKPfh8kGScIF28GLLTYtSSppoqGUZwarYmW5G8lslo8qAeeJ1IXW_h7wjmYUjoBq6-gWzxLxZt63vXo68O2C22V-2lXmzTPPBltwIapsw4BUo-NFDaiv-RHUPNSUw7rcPNiEN6BLZypii8CiGVhlhqVVAq7McJckXUdlaTX4G-6LA7bDi5L79_Qj

In [22]:
#Queries needed 

CSST_queries = ['CSST_Dollar Exposure_SD_TC','CSST_PnL_SD_TC']

queries = CSST_queries

In [23]:
#Download one by one the Queries and put the data into a df
dict_df = {}

for queries_names in queries:
    url_1 = "https://cargill-prod-aer.fisglobal.com//"
    # url_1 = "https://cargill-uat-aer.fisglobal.com//"
    url_2 = '/AdaptivFusionInvestigationService/api/Calculations/RiskService/Calculate/Results/{}'.format(queries_names)
    myUrl = url_1 + url_2
    head = {'Authorization': 'Bearer {}'.format(token)}

    response = requests.get(myUrl, headers=head, verify=True)
    #print('Server status :',response.status_code)
    print(f'Server status:{queries_names} {response.status_code}')
    
    result = str(response.content, 'utf-8')
    data = StringIO(result)
    data_df = pd.read_csv(data)
     
    dict_df[queries_names] = data_df

Server status:CSST_Dollar Exposure_SD_TC 200
Server status:CSST_PnL_SD_TC 200


In [24]:
#Saving Dollar Exposure TC df as .csv file
dollar_exposure_tc = dict_df['CSST_Dollar Exposure_SD_TC']
dollar_exposure_tc.to_csv(Dollar_Exposure_TC, index=False)
print('Dollar Exposure TC Saved')

Dollar Exposure TC Saved


In [25]:
#Saving Dollar Exposure TC df as .csv file
pnl_tc = dict_df['CSST_PnL_SD_TC']
pnl_tc.to_csv(PnL_TC, index=False)
print('PnL TC Saved')

PnL TC Saved
